# NFL Game Simulator

In [59]:
import numpy as np
import random
import pandas as pd
import math

from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import Imputer
from xgboost import XGBClassifier, XGBRegressor

from sklearn.metrics import accuracy_score, classification_report, explained_variance_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from IPython.display import clear_output
import pickle


In [60]:

global CurrentSeason, CurrentWeek
global SimulateWeek, SimulateSeason
global MaxWeek, MaxSeason

CurrentWeek = 11
CurrentSeason = 2018


SimulateWeek = 5
SimulateSeason = 2018

MaxWeek = 10
MaxSeason = 2018
TryToReadModels = False

playtypes = ['PASS','FUMBLE','INTERCPT','KOFF','PUNT','RUSH','SACK','SAFETY','TACKLE']
predictplays = ['PASS','KOFF','PUNT','RUSH','FGXP']

In [61]:
### define function to merge play data ###
def mergedata(playdata, type):
    file1 = "data/nfl_18/{playtype}.csv".format(playtype=type)
    file2 = "data/nfl_00-17/{playtype}.csv".format(playtype=type) 
    
    currentdata = pd.read_csv(file1)
    currentdata2 = pd.read_csv(file2)
    
    currentdata = pd.concat([currentdata,currentdata2])
    currentdata.drop_duplicates(keep='first')
    df = playdata.merge(currentdata, left_on='pid',right_on='pid',how='left').copy()
    return df

def convertsec(row):
    minute = int((row['min'])*60)
    second = int(row['sec'])
    return int(900-minute-second)

def player1(row):
    if row['type'] == 'RUSH':
        return row['bc']
    elif row['type'] == 'PASS':
        return row['psr']
    elif row['type'] == 'KOFF':
        return row['kicker']
    elif row['type'] == 'PUNT':
        return row['punter']
    elif row['type'] == 'FGXP':
        return row['fkicker']
    
def player2(row):
    if row['type'] == 'RUSH':
        return ''
    elif row['type'] == 'PASS':
        return row['trg']
    elif row['type'] == 'KOFF':
        return ''
    elif row['type'] == 'PUNT':
        return ''
    elif row['type'] == 'FGXP':
        return ''
    
def seasonnum(row):
    season = SimulateSeason - int(row['seas'])
    return season

def yardsgain(row):
    if not pd.isna(row['yds_x']):
        return row['yds_x']
    elif not pd.isna(row['yds_y']):
        return row['yds_y']
    else:
        return 0
 
def half(row):
    if row['qtr'] in (1,2):  
        return 1
    else:
        return 2
    
def create_encoder(df, features):
    '''returns an encoder for the passed dataframe'''
    cat_features = list(df[features].select_dtypes(include='object').columns)
    num_features = list(df[features].select_dtypes(exclude='object').columns)
    
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(df[cat_features])
    
    return enc


def create_dummies(df, features, encoder):
    '''returns a dataframe with dummies created by the specified encoder'''
    cat_features = list(df[features].select_dtypes(include='object').columns)
    num_features = list(df[features].select_dtypes(exclude='object').columns)
    
    dummy = pd.DataFrame(encoder.transform(df[cat_features]).toarray(), index=df.index)
    dummy.columns = encoder.get_feature_names()
    
    return pd.concat([df[num_features], dummy], axis=1)

### drop variables ###
def dropvars(df, dropvars):
    return df.drop(dropvars,axis=1).copy()



# Read in play data function###

In [62]:
### read in all plays ###

def readplays(season, curweek):
    seasonback = 0
    # Read in play by play data and merge with game information #
    playdata = pd.read_csv('data/nfl_00-17/play.csv')
    playdata1 = pd.read_csv('data/nfl_18/play.csv')
    playdata = pd.concat([playdata,playdata1])
    playdata.drop_duplicates(keep='first')
    playdata1 = None

    ### merge with game level data - IDEALLY WOULD LIKE TO USE GAME DATASET BUT THERE ARE FIELDS ON THERE NOT AVAILABLE FOR CURRENT WEEK ###
    gamedata = pd.read_csv('data/nfl_18/SCHEDULE.csv')

    playdata = playdata.merge(gamedata, left_on='gid', right_on='gid', how='left')

    for playtype in predictplays:
        playdata = mergedata(playdata, playtype)
    
    
    ### FILTER TO LAST TWO SEASONS ####
    playdata = playdata[((playdata['seas'] >= season-seasonback) & (playdata['seas'] <= season-1)) | ((playdata['seas'] == season) & (playdata['wk'] < curweek))]
    return playdata


In [63]:
def FeatureEdit(playdata):
    # perform feature transformations and create new features #
    
    ### convert min and sec to sec ###
    playdata['sec'] = playdata.apply (lambda row: convertsec(row),axis=1)
    playdata['half'] = playdata.apply (lambda row: half(row),axis=1)
    playdata = playdata.drop(['min'],axis=1)

    ### find time of play ###
    playdata['srsnum'] = (playdata['off'] != playdata['off'].shift(1)).astype(int).cumsum()

    playdata['playtime'] = np.nan 

    playdata.playtime = playdata.groupby(['gid','half','srsnum'])['sec'].diff(periods=-1)
    playdata['playtime'] = playdata['playtime']*-1
    playdata = playdata.drop(['srsnum','half'],axis=1)

    ### convert seas to this seas - seas ###
    playdata['seas'] = playdata.apply (lambda row: seasonnum(row),axis=1)

    ### filter play dataset to callable plays ###
    playdata = playdata.loc[playdata['type'].isin(predictplays)]

    ### Define play1 and player 2 ###
    playdata['player1'] = playdata.apply (lambda row: player1(row),axis=1)
    playdata['player2'] = playdata.apply (lambda row: player2(row),axis=1)
    playdata = playdata.drop(['bc','psr','kicker','punter','trg','fkicker'],axis=1)
    return playdata
    

# Define train model function

In [272]:
def trainmodel(df, features, target, file, clf, clfE, modeltype):
    outputstatus = False
    ### define target ###
    #should be type-sg-nh (type, shotgun, no huddle)
    ### for now drop complex paly types ###
    y = df[target].copy()
    X = df.drop([target],axis=1).copy()
    X = X.drop(['sg','nh','loc','dir'],axis=1)
    if file not in ("PuntModel","KoffModel","FGXPModel"):
        X = X.drop(['player1','player2'],axis=1)
    else:
        X = X.drop(['player2'],axis=1)
    if file not in('PassRunModel','PlayModel','TimeModel'):
        X = X.drop(['type'],axis=1)

    ### encode categorical variables ###
    if outputstatus == True:
        print("Status: Encoding")
    encoder = create_encoder(X, features)
    X = create_dummies(X, features, encoder)
    
    ### save encoder ###
    if outputstatus == True:    
        print("Status: Saving Encoder")    
    filename = 'encoders/{filename}_Encoder_S{season}_W{week}.sav'.format(filename=file,season=SimulateSeason,week=SimulateWeek)
    pickle.dump(encoder, open(filename, 'wb')) 

    
    ### split to test train ###
    if outputstatus == True:    
        print("Status: Split Test Train")    
    X_train, X_test, y_train, y_test = train_test_split(X.copy(), y.copy(), test_size=0.33, random_state=42)    
 
    ### Train Model ###
    if outputstatus == True:    
        print("Status: Training Model")    
    Model = clf.fit(X_train, y_train, sample_weight=None)

    ### save model ###
    if outputstatus == True:    
        print("Status: Saving Model")        
    filename = 'models/{filename}_S{season}_W{week}.sav'.format(filename=file,season=SimulateSeason,week=SimulateWeek)
    pickle.dump(Model, open(filename, 'wb'))

    ### score dataset
    if outputstatus == True:    
        print("Status: Test Predictions")        
    predictions = Model.predict(X_test)
    if outputstatus == True:    
        if modeltype == 'Classification':
            ### show predictions ###
            print("Status: Score Test Dataset")        
            AccuracyRate = accuracy_score(y_test, predictions)
            print("Accuracy: %.2f%%" % (AccuracyRate * 100.0))

        else:
            print("Status: Score Test Dataset")        
            print("RMSE: %.2f"% math.sqrt(np.mean((predictions - y_test) ** 2)))
            
    if modeltype == 'Regressor':       
        ### train error model ###
        # compute the prediction error vector on the validation set
        validation_error = abs(predictions - y_test)
        ErrorModel = clfE.fit(X_test, validation_error)
        filename = 'ErrorModels/{filename}_S{season}_W{week}.sav'.format(filename=file,season=SimulateSeason,week=SimulateWeek)
        pickle.dump(ErrorModel, open(filename, 'wb'))
        return Model, encoder, ErrorModel
    else: 
        return Model, encoder


# Read in Play Data

In [273]:
### read ing and transform data
playdata = readplays(SimulateSeason, SimulateWeek)
playdata = FeatureEdit(playdata)
#print(playdata.head(20))
#playdata.to_csv('test.csv')

## Define model specific variables ##

In [274]:

### drop unecessary variables
dropvarsPlaySelect=['dseq','len','timo','timd','zone','fd','pts','tck'
 ,'sk','pen','ints','fum','saf','blk','yds_x','comp'
 ,'succ_x','spk','dfb','yds_y','succ_y','kne','kgro','knet','ktb','kr','kry'
 ,'pgro','pnet','ptb','pr','pry','pfc','pid','gid', 'wk','date','v','h'
 ,'fgxp','dist','good','playtime']

dropvarsPassRunSuccess=['dseq','len','timo','timd','zone','fd','pts','tck'
 ,'sk','pen','ints','fum','saf','blk'
 ,'succ_x','spk','dfb','succ_y','kne','kgro','knet','ktb','kr','kry'
 ,'pgro','pnet','ptb','pr','pry','pfc','pid','gid', 'wk','date','v','h'
 ,'fgxp','dist','good','playtime']


dropvarsPuntSuccess=['dseq','len','timo','timd','zone','fd','pts','tck'
 ,'sk','pen','ints','fum','saf','blk','yds_x','comp'
 ,'succ_x','spk','dfb','yds_y','succ_y','kne','kgro','knet','ktb','kr','kry'
 ,'pgro','ptb','pr','pry','pfc','pid','gid','wk','date','v','h'
 ,'fgxp','dist','good','playtime']

dropvarsKoffSuccess=['dseq','len','timo','timd','zone','fd','pts','tck'
 ,'sk','pen','ints','fum','saf','blk','yds_x','comp'
 ,'succ_x','spk','dfb','yds_y','succ_y','kne','kgro','ktb','kr','kry'
 ,'pgro','pnet','ptb','pr','pry','pfc','pid','gid', 'wk','date','v','h'
 ,'fgxp','dist','good','playtime']

dropvarsFgxpSuccess=['dseq','len','timo','timd','zone','fd','pts','tck'
 ,'sk','pen','ints','fum','saf','blk','yds_x','comp'
 ,'succ_x','spk','dfb','yds_y','succ_y','kne','kgro','knet','ktb','kr','kry'
 ,'pgro','pnet','ptb','pr','pry','pfc','pid','gid', 'wk','date','v','h'
 ,'fgxp','playtime']

dropvarsPlayTime=['dseq','len','timo','timd','zone','fd','pts','tck'
 ,'sk','pen','ints','fum','saf','blk','yds_x','comp'
 ,'succ_x','spk','dfb','yds_y','succ_y','kne','kgro','knet','ktb','kr','kry'
 ,'pgro','pnet','ptb','pr','pry','pfc','pid','gid', 'wk','date','v','h'
 ,'fgxp']



### define features ###
featuresPlay    = ['seas','off','def','qtr','sec','ptso','ptsd','dwn','ytg','yfog','day','stad','surf']
featuresPassRun = ['seas','off','def','type','qtr','sec','ptso','ptsd','dwn','ytg','yfog','day','stad','surf']
featuresPunt    = ['seas','off','def','qtr','sec','ptso','ptsd','dwn','ytg','yfog','day','stad','surf','player1']
featuresKOFF    = ['seas','off','def','qtr','sec','ptso','ptsd','dwn','ytg','yfog','day','stad','surf','player1']
featuresFGXP    = ['seas','off','def','qtr','sec','ptso','ptsd','dwn','ytg','yfog','day','stad','surf','dist','player1']
featuresPlayTime= ['seas','off','def','type','qtr','sec','ptso','ptsd','dwn','ytg','yfog','day','stad','surf']

# Train Models

In [275]:
### play selection model ###
try:
    if TryToReadModels == False:
        raise
    filename = 'models/{filename}_S{season}_W{week}.sav'.format(filename='PlayModel',season=SimulateSeason,week=SimulateWeek)
    PlayModel = open(filename,'rb')
    PlayModel = pickle.load(PlayModel)
    filename = 'encoders/{filename}_Encoder_S{season}_W{week}.sav'.format(filename='PlayModel',season=SimulateSeason,week=SimulateWeek)
    PlayEncoder = open(filename,'rb')
    PlayEncoder = pickle.load(PlayEncoder)
    print("Model Imported Successfully")

except:  
    clfclassM = XGBClassifier(learning_rate =0.3,
                     n_estimators=500,
                     max_depth=3,
                     min_child_weight=1,
                     gamma=0,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     objective= 'multi:softmax',
                     nthread=4,
                     scale_pos_weight=1,
                     seed=27)
    playdataPlaySelect = dropvars(playdata, dropvarsPlaySelect).copy()
    PlayModel, PlayEncoder = trainmodel(playdataPlaySelect, featuresPlay, 'type', "PlayModel", clfclassM, clfclassM, 'Classification')

In [276]:
### Field Goal Extra Point Model ###
try:
    if TryToReadModels == False:
        raise    
    filename = 'models/{filename}_S{season}_W{week}.sav'.format(filename='FGXPModel',season=SimulateSeason,week=SimulateWeek)
    FgxpModel = open(filename,'rb')
    FgxpModel = pickle.load(FgxpModel)
    filename = 'encoders/{filename}_Encoder_S{season}_W{week}.sav'.format(filename='FGXPModel',season=SimulateSeason,week=SimulateWeek)
    FgxpEncoder = open(filename,'rb')
    FgxpEncoder = pickle.load(FgxpEncoder)
    print("Model Imported Successfully")
except:
    clfclassB = XGBClassifier(learning_rate =0.3,
                     n_estimators=500,
                     max_depth=3,
                     min_child_weight=1,
                     gamma=0,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     objective= 'binary:logistic',
                     nthread=4,
                     scale_pos_weight=1,
                     seed=27)
    playdataFGXP = dropvars(playdata, dropvarsFgxpSuccess).copy()
    playdataFGXP = playdataFGXP.loc[playdataFGXP['type'].isin(['FGXP'])]
    playdataFGXP['good'] = playdataFGXP['good'].astype(int)
    FgxpModel, FgxpEncoder = trainmodel(playdataFGXP, featuresFGXP, 'good', "FGXPModel", clfclassB, clfclassB, 'Classification')

In [277]:
### Pass Run Model ###
try:
    if TryToReadModels == False:
        raise    
    filename = 'models/{filename}_S{season}_W{week}.sav'.format(filename='PassRunModel',season=SimulateSeason,week=SimulateWeek)
    PassRunModel = open(filename,'rb')
    PassRunModel = pickle.load(PassRunModel)
    filename = 'encoders/{filename}_Encoder_S{season}_W{week}.sav'.format(filename='PassRunModel',season=SimulateSeason,week=SimulateWeek)
    PassRunEncoder = open(filename,'rb')
    PassRunEncoder = pickle.load(PassRunEncoder)
    print("Model Imported Successfully")
except:
    clfregress1 = XGBRegressor(max_depth = 5,
                        min_child_weight = 10,
                        subsample = 0.5,
                        colsample_bytree = 0.6,
                        objective = 'reg:linear',
                        num_estimators = 1000,
                        learning_rate = 0.3,
                        nthread=4)
    clfregress1E = XGBRegressor(max_depth = 5,
                        min_child_weight = 10,
                        subsample = 0.5,
                        colsample_bytree = 0.6,
                        objective = 'reg:linear',
                        num_estimators = 1000,
                        learning_rate = 0.3,
                        nthread=4)
    playdataPassRun = dropvars(playdata, dropvarsPassRunSuccess).copy()
    playdataPassRun = playdataPassRun.loc[playdataPassRun['type'].isin(['PASS','RUSH'])]
    playdataPassRun['YardsGain'] = playdataPassRun.apply (lambda row: yardsgain(row),axis=1)
    playdataPassRun = dropvars(playdataPassRun, ['yds_x','comp','yds_y'])
    PassRunModel, PassRunEncoder, PassRunErrorModel = trainmodel(playdataPassRun, featuresPassRun, 'YardsGain', "PassRunModel", clfregress1, clfregress1E, 'Regressor')

In [278]:
### Punt Model ###
try:
    if TryToReadModels == False:
        raise   
    filename = 'models/{filename}_S{season}_W{week}.sav'.format(filename='PuntModel',season=SimulateSeason,week=SimulateWeek)
    PuntModel = open(filename,'rb')
    PuntModel = pickle.load(PuntModel)
    filename = 'encoders/{filename}_Encoder_S{season}_W{week}.sav'.format(filename='PuntModel',season=SimulateSeason,week=SimulateWeek)
    PuntEncoder = open(filename,'rb')
    PuntEncoder = pickle.load(PuntEncoder)
    print("Model Imported Successfully")
except:
    clfregress2 = XGBRegressor(max_depth = 5,
                        min_child_weight = 10,
                        subsample = 0.5,
                        colsample_bytree = 0.6,
                        objective = 'reg:linear',
                        num_estimators = 1000,
                        learning_rate = 0.3,
                        nthread=4)
    clfregress2E = XGBRegressor(max_depth = 5,
                        min_child_weight = 10,
                        subsample = 0.5,
                        colsample_bytree = 0.6,
                        objective = 'reg:linear',
                        num_estimators = 1000,
                        learning_rate = 0.3,
                        nthread=4)
    playdataPunt = dropvars(playdata, dropvarsPuntSuccess).copy()
    playdataPunt = playdataPunt.loc[playdataPunt['type'].isin(['PUNT'])]
    PuntModel, PuntEncoder, PuntErrorModel = trainmodel(playdataPunt, featuresPunt, 'pnet', "PuntModel", clfregress2, clfregress2E, 'Regressor')

In [279]:
try:
    if TryToReadModels == False:
        raise    
    filename = 'models/{filename}_S{season}_W{week}.sav'.format(filename='KoffModel',season=SimulateSeason,week=SimulateWeek)
    KoffModel = open(filename,'rb')
    KoffModel = pickle.load(KoffModel)
    filename = 'encoders/{filename}_Encoder_S{season}_W{week}.sav'.format(filename='KoffModel',season=SimulateSeason,week=SimulateWeek)
    KoffEncoder = open(filename,'rb')
    KoffEncoder = pickle.load(KoffEncoder)
    print("Model Imported Successfully")
except:
    clfregress3 = XGBRegressor(max_depth = 5,
                        min_child_weight = 10,
                        subsample = 0.5,
                        colsample_bytree = 0.6,
                        objective = 'reg:linear',
                        num_estimators = 1000,
                        learning_rate = 0.3,
                        nthread=4)
    clfregress3E = XGBRegressor(max_depth = 5,
                        min_child_weight = 10,
                        subsample = 0.5,
                        colsample_bytree = 0.6,
                        objective = 'reg:linear',
                        num_estimators = 1000,
                        learning_rate = 0.3,
                        nthread=4)
    playdataKOFF = dropvars(playdata, dropvarsKoffSuccess).copy()
    playdataKOFF = playdataKOFF.loc[playdataKOFF['type'].isin(['KOFF'])]
    playdataKOFF = playdataKOFF[playdataKOFF['player1'].notnull()]
    KoffModel, KoffEncoder, KoffErrorModel = trainmodel(playdataKOFF, featuresKOFF, 'knet', "KoffModel", clfregress3, clfregress3E, 'Regressor')

In [280]:
### Punt Model ###
try:
    if TryToReadModels == False:
        raise    
    filename = 'models/{filename}_S{season}_W{week}.sav'.format(filename='TimeModel',season=SimulateSeason,week=SimulateWeek)
    TimeModel = open(filename,'rb')
    TimeModel = pickle.load(TimeModel)
    filename = 'encoders/{filename}_Encoder_S{season}_W{week}.sav'.format(filename='TimeModel',season=SimulateSeason,week=SimulateWeek)
    TimeEncoder = open(filename,'rb')
    TimeEncoder = pickle.load(TimeEncoder)
    print("Model Imported Successfully")
except:
    clfregress4 = XGBRegressor(max_depth = 5,
                        min_child_weight = 10,
                        subsample = 0.5,
                        colsample_bytree = 0.6,
                        objective = 'reg:linear',
                        num_estimators = 1000,
                        learning_rate = 0.3,
                        nthread=4)
    clfregress4E = XGBRegressor(max_depth = 5,
                        min_child_weight = 10,
                        subsample = 0.5,
                        colsample_bytree = 0.6,
                        objective = 'reg:linear',
                        num_estimators = 1000,
                        learning_rate = 0.3,
                        nthread=4)
    playdataTime = playdata[(playdata['playtime'] > 0)].copy()
    playdataTime = playdataTime.dropna(subset=['playtime'])
    playdataTime = dropvars(playdataTime, dropvarsPlayTime)
    TimeModel, TimeEncoder, TimeErrorModel = trainmodel(playdataTime, featuresPlayTime, 'playtime', "TimeModel", clfregress4, clfregress4E, 'Regressor')


## Make models and encoders global

In [281]:
global featuresPlay, PlayModel, PlayErrorModel, PlayEncoder
global featuresPassRun, PassRunModel, PassRunErrorModel, PassRunEncoder
global featuresPunt, PuntModel, PuntErrorModel, PuntEncoder
global featuresKOFF, KoffModel, KoffErrorModel, KoffEncoder
global featuresFGXP, FgxpModel, FgxpErrorModel, FgxpEncoder
global featuresPlayTime, TimeModel, TimeErrorModel, TimeEncoder


# Define simulator functions

In [316]:
def coinflip():
    ### coin flip ###
    FlipPct = random.random()
    if FlipPct < 0.5:
        ## Away picks ##
        BallPos = 0
    else:
        ## Home picks
        BallPos = 1
    ## add prob of deferring ##
    return BallPos


def runextrapoint(Offense, Defense, GameQtr, HomeScore, AwayScore):
    ExtPPct = random.random()
    if ExtPPct > 0.05:
        PointsAdded = 1
    else:
        PointsAdded = 0
    return PointsAdded


def updatetime(GameTime, TimeChange, GameQtr):
    ## update quarter and time ##
    if GameTime + TimeChange < 900:
        GameTime += TimeChange
    else:
        GameQtr+=1
        GameTime = 0
    return GameTime, GameQtr

def changepossession(BallPos, Offense, Defense, HomeTeam, AwayTeam, HomePunter, AwayPunter, HomeQB, AwayQB, HomeKicker, AwayKicker, HomeFGKicker, AwayFGKicker):
    if BallPos == 1:
        BallPos = 0
    else:
        BallPos = 1
    if BallPos == 1:
        Offense = HomeTeam
        Defense = AwayTeam
        Kicker = HomeKicker
        FGKicker = HomeFGKicker
        Punter = HomePunter
        QB = HomeQB
    else:
        Offense = AwayTeam
        Defense = HomeTeam
        Kicker = AwayKicker
        FGKicker = AwayFGKicker
        Punter = AwayPunter
        QB = AwayQB        
    GameDown = 1
    GameYrsToGo = 10
    return BallPos, Offense, Defense, GameDown, GameYrsToGo, Kicker, FGKicker, Punter, QB


############################
### these ones need work ###
############################
def chooseplay(Offense, Defense, GameTime, GameQtr, GameDown, GameYrsToGo, CurrentYard, HomeScore, AwayScore, BallPos, Field, Surface, day):
    if BallPos == 1:
        currentstatus = [0, Offense, Defense, GameQtr, GameTime, HomeScore, AwayScore, GameDown, GameYrsToGo, CurrentYard, Field, Surface, day]
    else:
        currentstatus = [0, Offense, Defense, GameQtr, GameTime, AwayScore, HomeScore, GameDown, GameYrsToGo, CurrentYard, Field, Surface, day]
    
    ### convert to dataframe
    currentstatus = pd.DataFrame(np.array(currentstatus).reshape(1,13), columns = list(featuresPlay))

    currentstatus[['seas','qtr','sec','ptso','ptsd','dwn','ytg','yfog']] = currentstatus[['seas','qtr','sec','ptso','ptsd','dwn','ytg','yfog']].apply(pd.to_numeric)

    ### get dummies ###
    currentstatus = create_dummies(currentstatus, featuresPlay, PlayEncoder)
    ### apply play predict model ###
    PlayType = PlayModel.predict(currentstatus)
    PlayType=PlayType[0]
    
    PlayPlayer1 = "Chase" 
    PlayPlayer2 = "Patty"

    return PlayType, PlayPlayer1, PlayPlayer2
                
def playoutcome(Offense, Defense, PlayType, GameTime, GameQtr, HomeScore, AwayScore, GameDown, GameYrsToGo, CurrentYard, Field, Surface, day, FGKicker, Kicker, Punter):
    # Offense probability of yards gain/lost and deviation #
    # Defense probability of stopping play type #
    ### initialize field goal make ##
    FieldGoalMake = False
    if PlayType == "FGXP":
        PlayPlayer1 = FGKicker
        ### add logic for kicker accuracy ###
        currentstatus = [0, Offense, Defense, GameQtr, GameTime, HomeScore, AwayScore, GameDown, GameYrsToGo, CurrentYard, Field, Surface, day, int(100-GameYrsToGo+10+15), PlayPlayer1]
        currentstatus = pd.DataFrame(np.array(currentstatus).reshape(1,15), columns = list(featuresFGXP))
        currentstatus[['seas','qtr','sec','ptso','ptsd','dwn','ytg','yfog','dist']] = currentstatus[['seas','qtr','sec','ptso','ptsd','dwn','ytg','yfog','dist']].apply(pd.to_numeric)
        currentstatus = create_dummies(currentstatus, featuresFGXP, FgxpEncoder)
        ### apply play predict model ###
        FGXPSuccess = FgxpModel.predict_proba(currentstatus)[0,1]
        proba = np.random.uniform(0,1,1)[0]
        if proba <= FGXPSuccess:
            FieldGoalMake = True
        YardChange = 0
            
    elif PlayType == "PUNT":
        PlayPlayer1 = Punter
        currentstatus = [0, Offense, Defense, GameQtr, GameTime, HomeScore, AwayScore, GameDown, GameYrsToGo, CurrentYard, Field, Surface, day, PlayPlayer1]
        currentstatus = pd.DataFrame(np.array(currentstatus).reshape(1,14), columns = list(featuresPunt))
        currentstatus[['seas','qtr','sec','ptso','ptsd','dwn','ytg','yfog']] = currentstatus[['seas','qtr','sec','ptso','ptsd','dwn','ytg','yfog']].apply(pd.to_numeric)
        currentstatus = create_dummies(currentstatus, featuresPunt, PuntEncoder)
        ### apply play predict model ###
        PuntYards = PuntModel.predict(currentstatus)
        PuntError = PuntErrorModel.predict(currentstatus)
        #PuntError = [20]
        #PuntYards = np.random.normal(PuntYards[0], max(1,PuntError[0]),1)
        PuntYards = np.random.poisson(max(PuntYards[0],1), 1)
        PuntYards = int(round(PuntYards[0]))
        YardChange = CurrentYard - max(0,100 - (CurrentYard + PuntYards))

    elif PlayType == "KOFF":
        PlayPlayer1 = Kicker
        currentstatus = [0, Offense, Defense, GameQtr, GameTime, HomeScore, AwayScore, GameDown, GameYrsToGo, CurrentYard, Field, Surface, day, PlayPlayer1]
        currentstatus = pd.DataFrame(np.array(currentstatus).reshape(1,14), columns = list(featuresKOFF))
        currentstatus[['seas','qtr','sec','ptso','ptsd','dwn','ytg','yfog']] = currentstatus[['seas','qtr','sec','ptso','ptsd','dwn','ytg','yfog']].apply(pd.to_numeric)
        currentstatus = create_dummies(currentstatus, featuresKOFF, KoffEncoder)
        ### apply play predict model ###
        KickYards = KoffModel.predict(currentstatus)
        KickError = KoffErrorModel.predict(currentstatus)
        #KickError = [20]
        #KickYards = np.random.normal(KickYards[0], max(1,KickError[0]),1)
        KickYards = np.random.poisson(max(1,KickYards[0]), 1)        
        KickYards = int(round(KickYards[0]))
        YardChange = CurrentYard - max(0,100 - (CurrentYard + KickYards))
        
    elif PlayType in ("PASS","RUSH"):
        currentstatus = [0, Offense, Defense, PlayType, GameQtr, GameTime, HomeScore, AwayScore, GameDown, GameYrsToGo, CurrentYard, Field, Surface, day]
        currentstatus = pd.DataFrame(np.array(currentstatus).reshape(1,14), columns = list(featuresPassRun))
        currentstatus[['seas','qtr','sec','ptso','ptsd','dwn','ytg','yfog']] = currentstatus[['seas','qtr','sec','ptso','ptsd','dwn','ytg','yfog']].apply(pd.to_numeric)
        currentstatus = create_dummies(currentstatus, featuresPassRun, PassRunEncoder)
        ### apply play predict model ###
        PassRunYards = PassRunModel.predict(currentstatus)
        PassRunYards2 = PassRunYards
        PassRunError = PassRunErrorModel.predict(currentstatus)
        #PassRunError = [20]
        #PassRunYards = np.random.normal(PassRunYards[0], max(1,PassRunError[0]),1)
        PassRunYards = np.random.poisson(max(1,PassRunYards[0]),1)
        PassRunYards=max(-20, int(round(PassRunYards[0])))
        YardChange = PassRunYards
        #print("Mean: {mean}, Dev: {dev}, Final: {yc}".format(mean=PassRunYards2,dev=PassRunError[0],yc=YardChange))

    ### find time of play ###
    currentstatus = [0, Offense, Defense, PlayType, GameQtr, GameTime, HomeScore, AwayScore, GameDown, GameYrsToGo, CurrentYard, Field, Surface, day]
    currentstatus = pd.DataFrame(np.array(currentstatus).reshape(1,14), columns = list(featuresPlayTime))
    currentstatus[['seas','qtr','sec','ptso','ptsd','dwn','ytg','yfog']] = currentstatus[['seas','qtr','sec','ptso','ptsd','dwn','ytg','yfog']].apply(pd.to_numeric)
    currentstatus = create_dummies(currentstatus, featuresPlayTime, TimeEncoder)
    ### apply play predict model ###
    TimeChange = TimeModel.predict(currentstatus)
    #TimeError = TimeErrorModel.predict(currentstatus)
    #TimeChange = np.random.normal(TimeChange[0], max(1,TimeError[0]),1)
    TimeChange=max(int(round(TimeChange[0])),3)

    #### old time change logic ###
    #TimeChange = np.random.normal(40, 20, 1)
    #TimeChange = round(TimeChange[0])
    #TimeChange = max(45+9,TimeChange)
    
    return YardChange, TimeChange, FieldGoalMake
    

In [317]:
def findplayers(ThisSeason, ThisWeek, HomeTeam, AwayTeam):
    ### find turf ###
    GameInfo = pd.read_csv('data/nfl_18/SCHEDULE.csv')
    GameInfo = GameInfo[(GameInfo.seas == ThisSeason) & (GameInfo.wk == ThisWeek) & (GameInfo.v == AwayTeam) & (GameInfo.h == HomeTeam)]
    #PlayTurf = GameInfo.iloc[0]['surf']
    PlayTime = GameInfo.iloc[0]['date']

    Field = GameInfo.iloc[0]['stad']
    Surface = GameInfo.iloc[0]['surf']

    ### find punter ###
    PunterInfoHome = playdata[(playdata.type == 'PUNT') & (playdata['def'] == HomeTeam)].copy()
    try:
        HomePunter = PunterInfoHome['player1'].value_counts().idxmax()
    except:
        HomePunter = 'unkwn'
    #print("Home Punter: " + HomePunter)

    PunterInfoAway = playdata[(playdata.type == 'PUNT') & (playdata['def'] == AwayTeam)].copy()
    try:
        AwayPunter = PunterInfoAway['player1'].value_counts().idxmax()
    except:
        AwayPunter = 'unkwn'
    #print("Away Punter: " + AwayPunter)

    ### find quarterback ###
    QBInfoHome = playdata[(playdata.type == 'PASS') & (playdata['off'] == HomeTeam)].copy()
    try:
        HomeQB = QBInfoHome['player1'].value_counts().idxmax()
    except:
        HomeQB = 'unkwn'
    #print("Home QB: " + HomeQB)

    QBInfoAway = playdata[(playdata.type == 'PASS') & (playdata['off'] == AwayTeam)].copy()
    try:
        AwayQB = QBInfoAway['player1'].value_counts().idxmax()
    except:
        AwayQB = 'unkwn'
    #print("Away QB: " + AwayQB)

    ### find kickoff  ###
    KoffInfoHome = playdata[(playdata.type == 'KOFF') & (playdata['def'] == HomeTeam)].copy()
    try:
        HomeKicker = KoffInfoHome['player1'].value_counts().idxmax()
    except:
        HomeKicker = 'unkwn'
    #print("Home Kicker: " + HomeKicker)

    KoffInfoAway = playdata[(playdata.type == 'KOFF') & (playdata['def'] == AwayTeam)].copy()
    try:
        AwayKicker = KoffInfoAway['player1'].value_counts().idxmax()
    except:
        AwayKicker = 'unkwn'
    #print("Away Kicker: " + AwayKicker)

    ### find FG kickoff  ###
    FGXPInfoHome = playdata[(playdata.type == 'FGXP') & (playdata['off'] == HomeTeam)].copy()
    try:
        HomeFGKicker = FGXPInfoHome['player1'].value_counts().idxmax()
    except:
        HomeFGKicker = 'unkwn'
    #print("Home FG Kicker: " + HomeFGKicker)

    FGXPInfoAway = playdata[(playdata.type == 'FGXP') & (playdata['off'] == AwayTeam)].copy()
    try:
        AwayFGKicker = FGXPInfoAway['player1'].value_counts().idxmax()
    except:
        AwayFGKicker = 'unkwn'
    #print("Away FG Kicker: " + AwayFGKicker)

    return Field, Surface, HomePunter, AwayPunter, HomeQB, AwayQB, HomeKicker, AwayKicker, HomeFGKicker, AwayFGKicker

# Define simulator function

In [318]:

def simulate(HomeTeam, AwayTeam, Field, Surface, day, HomePunter, AwayPunter, HomeQB, AwayQB, HomeKicker, AwayKicker, HomeFGKicker, AwayFGKicker, BallPos):
    ### initialize variables ###
    PrintGameLog = False
    GameTime = 0
    GameQtr = 1
    GameDown = 1
    GameYrsToGo = 10
    HomeScore = 0
    AwayScore = 0
    CurrentYard = 25
    TimeChange = 0
    
    while True:
        ### initialize game ###
        if (GameTime==0 and GameQtr == 1):
            BallPos = coinflip()
            OrigBallPos = BallPos
            if BallPos == 1:
                Offense = HomeTeam
                Defense = AwayTeam
                Kicker = HomeKicker
                FGKicker = HomeFGKicker
                Punter = HomePunter
                QB = HomeQB
            else:
                Offense = AwayTeam
                Defense = HomeTeam
                Kicker = AwayKicker
                FGKicker = AwayFGKicker
                Punter = AwayPunter
                QB = AwayQB        
            ### kickoff ###
            CurrentYard = 25
                                           
            YardChange, TimeChange, FieldGoalMake = playoutcome(Offense, Defense, "KOFF", GameTime, GameQtr, HomeScore, AwayScore, GameDown, GameYrsToGo, CurrentYard, Field, Surface, day, FGKicker, Kicker, Punter) 
            ## update quarter and time ##
            GameTime, GameQtr = updatetime(GameTime, TimeChange, GameQtr)
            CurrentYard += YardChange 
        ### initialize half ###
        if (GameTime==0 and GameQtr == 3):
            if OrigBallPos == 1:
                Offense = AwayTeam
                Defense = HomeTeam
                Kicker = AwayKicker
                FGKicker = AwayFGKicker
                Punter = AwayPunter
                QB = AwayQB  
            else:
                Offense = HomeTeam
                Defense = AwayTeam
                Kicker = HomeKicker
                FGKicker = HomeFGKicker
                Punter = HomePunter
                QB = HomeQB
            ### kickoff ###
            CurrentYard = 25
            YardChange, TimeChange, FieldGoalMake = playoutcome(Offense, Defense, "KOFF", GameTime, GameQtr, HomeScore, AwayScore, GameDown, GameYrsToGo, CurrentYard, Field, Surface, day, FGKicker, Kicker, Punter) 
            ## update quarter and time ##
            GameTime, GameQtr = updatetime(GameTime, TimeChange, GameQtr)
            CurrentYard += YardChange 

        ## check to end game ##
        if (GameQtr > 4 or (GameQtr == 4 and GameTime >= 900)):
            break

        ### determine play based on situation ###
        PlayType, PlayPlayer1, PlayPlayer2 = chooseplay(Offense, Defense, GameTime, GameQtr, GameDown, GameYrsToGo, CurrentYard, HomeScore, AwayScore, BallPos, Field, Surface, day)
         
        
        if PrintGameLog == True:
            print("""Quarter: {quarter}, Time: {time} , Offence: {offence}
                        , Down: {down}, Yards Left: {yards}, Yard Line: {yardline}, Play Type: {playtype}
                        , Home Score: {HomeScore}, Away Score: {AwayScore}""".format(quarter=GameQtr
                                                                                            , time=GameTime
                                                                                            ,down=GameDown
                                                                                            ,offence=BallPos
                                                                                            ,yards=GameYrsToGo
                                                                                            ,yardline=CurrentYard
                                                                                            ,HomeScore=HomeScore
                                                                                            ,AwayScore=AwayScore
                                                                                            ,playtype=PlayType))                
        ### predict play outcome ###
        YardChange, TimeChange, FieldGoalMake = playoutcome(Offense, Defense, PlayType, GameTime, GameQtr, HomeScore, AwayScore, GameDown, GameYrsToGo, CurrentYard, Field, Surface, day, FGKicker, Kicker, Punter)

        ## if team is punting ##
        if PlayType == "PUNT":
            ## Change possession ##
            BallPos, Offense, Defense, GameDown, GameYrsToGo, Kicker, FGKicker, Punter, QB = changepossession(BallPos, Offense, Defense, HomeTeam, AwayTeam, HomePunter, AwayPunter, HomeQB, AwayQB, HomeKicker, AwayKicker, HomeFGKicker, AwayFGKicker)
            YardChange, TimeChange, FieldGoalMake = playoutcome(Offense, Defense, "PUNT", GameTime, GameQtr, HomeScore, AwayScore, GameDown, GameYrsToGo, CurrentYard, Field, Surface, day, FGKicker, Kicker, Punter)
            CurrentYard += YardChange                     
        ## if team kicked a field goal ##
        elif PlayType == "FGXP":
            if FieldGoalMake == True:
                if BallPos == 1:
                    HomeScore += 3
                else:
                    AwayScore += 3
            ## make or miss, switch possession ##
            CurrentYard = 25
            YardChange, TimeChange, FieldGoalMake = playoutcome(Offense, Defense, "KOFF", GameTime, GameQtr, HomeScore, AwayScore, GameDown, GameYrsToGo, CurrentYard, Field, Surface, day, FGKicker, Kicker, Punter)
            CurrentYard += YardChange            
            BallPos, Offense, Defense, GameDown, GameYrsToGo, Kicker, FGKicker, Punter, QB = changepossession(BallPos, Offense, Defense, HomeTeam, AwayTeam, HomePunter, AwayPunter, HomeQB, AwayQB, HomeKicker, AwayKicker, HomeFGKicker, AwayFGKicker)

        ## pass or run play ##
        else:     
            ### Did I score ###
            if CurrentYard + YardChange >= 100:
                ## kick extra point or go for it ##
                YardChange, TimeChange, FieldGoalMake = playoutcome(Offense, Defense, "FGXP", GameTime, GameQtr, HomeScore, AwayScore, 4, GameYrsToGo, 85, Field, Surface, day, FGKicker, Kicker, Punter) 
                if FieldGoalMake == True:
                    ExtraPoint = 1
                else:
                    ExtraPoint = 0
                if BallPos == 1:
                    HomeScore += 6 + ExtraPoint
                else:
                    AwayScore += 6 + ExtraPoint
                CurrentYard = 25    
                ### change possession ###
                BallPos, Offense, Defense, GameDown, GameYrsToGo, Kicker, FGKicker, Punter, QB = changepossession(BallPos, Offense, Defense, HomeTeam, AwayTeam, HomePunter, AwayPunter, HomeQB, AwayQB, HomeKicker, AwayKicker, HomeFGKicker, AwayFGKicker)
                
            ## if I did not score ##
            else:
                ## update yard and down ##
                CurrentYard += YardChange
                #first down#
                if GameYrsToGo - YardChange <= 0:
                    GameDown = 1
                    GameYrsToGo = 10
                ## next down ##
                else:
                    ## Change possession ##
                    if GameDown == 4:
                        BallPos, Offense, Defense, GameDown, GameYrsToGo, Kicker, FGKicker, Punter, QB = changepossession(BallPos, Offense, Defense, HomeTeam, AwayTeam, HomePunter, AwayPunter, HomeQB, AwayQB, HomeKicker, AwayKicker, HomeFGKicker, AwayFGKicker)
                    ### keep trying ###
                    else:
                        GameDown += 1
                        GameYrsToGo = GameYrsToGo - YardChange          


        ## update quarter and time ##
        GameTime, GameQtr = updatetime(GameTime, TimeChange, GameQtr)

        ## check to end game ##
        if (GameQtr > 4 or (GameQtr == 4 and GameTime >= 900)):
            break

    #print("Home Score: {HomeScore}, Away Score: {AwayScore}".format(HomeScore=HomeScore, AwayScore=AwayScore))
    return HomeScore, AwayScore


# Read in this week's games

In [321]:
### GET CURRENT WEEKS GAMES ###

gamedatapredict = pd.read_csv('data/nfl_18/SCHEDULE.csv')

gamedatapredict =  gamedatapredict[(gamedatapredict['seas']==SimulateSeason) & (gamedatapredict['wk']==SimulateWeek)]
'''
gamedatapredict =  gamedatapredict[(gamedatapredict['seas']==SimulateSeason) & (gamedatapredict['wk']==SimulateWeek) 
                                   & (gamedatapredict['h'] == 'NE')]'''
                                  
#print(gamedatapredict)



"\ngamedatapredict =  gamedatapredict[(gamedatapredict['seas']==SimulateSeason) & (gamedatapredict['wk']==SimulateWeek) \n                                   & (gamedatapredict['h'] == 'NE')]"

# For each game run N Simulations

In [322]:
CorrectPicks = 0
IncorrectPicks = 0

for index, row in gamedatapredict.iterrows():
    ### read in player data ###
    Field, Surface, HomePunter, AwayPunter, HomeQB, AwayQB, HomeKicker, AwayKicker, HomeFGKicker, AwayFGKicker = findplayers(SimulateSeason, SimulateWeek, row['h'], row['v'])

    ### define simulation stats ###
    SimulationCount = 10
    HomeWin = 0
    AwayWin = 0
    HomeScoreRun = 0
    AwayScoreRun = 0
    Tie = 0
    HomeScoreArray = []
    AwayScoreArray = []
    
    ### call simulator X times ###
    for itt in range(SimulationCount):
        curpct = round((itt+1)/SimulationCount*20)
        dots = '%'*curpct
        space = ' '*(20-curpct)
        print("|" + dots + space + "|  " + "Simulation: {simnum}/{simcount}".format(simnum=itt+1,simcount=SimulationCount),end="\r")
        HomeScore, AwayScore = simulate(row['h'],
                                        row['v'], 
                                        Field, 
                                        Surface, 
                                        row['day'], 
                                        HomePunter, 
                                        AwayPunter, 
                                        HomeQB, 
                                        AwayQB, 
                                        HomeKicker, 
                                        AwayKicker, 
                                        HomeFGKicker, 
                                        AwayFGKicker,
                                        itt)
        if HomeScore > AwayScore:
            HomeWin += 1
        elif HomeScore < AwayScore:
            AwayWin += 1
        else:
            Tie += 1
        HomeScoreRun += HomeScore
        AwayScoreRun += AwayScore
        HomeScoreArray.append(HomeScore)
        AwayScoreArray.append(AwayScore)
    
    ### find spread statistics ###
    mySpread = ((AwayScoreRun-HomeScoreRun)/SimulationCount)
    SpreadArray = [AwayS - HomeS for AwayS, HomeS in zip(AwayScoreArray, HomeScoreArray)]
    myModeSpread = max(set(SpreadArray), key=SpreadArray.count)
    
    ### find home and away team averages ###
    havg=HomeScoreRun/SimulationCount
    aavg=AwayScoreRun/SimulationCount
    
    ### logic to say if spread exists ###
    if MaxWeek >= SimulateWeek and MaxSeason >= SimulateSeason:
        SpreadData = pd.read_csv('data/nfl_18/GAME.csv')
        SpreadData = SpreadData[(SpreadData['seas']==SimulateSeason) & (SpreadData['wk']==SimulateWeek) & (SpreadData['h']==row['h'])]
        
        sprv = SpreadData.iloc[0]['sprv']*-1
        actual_spr = SpreadData.iloc[0]['ptsv'] - SpreadData.iloc[0]['ptsh']
        if actual_spr == sprv:
            CorrectPick = 'W'
        elif (mySpread <= sprv and actual_spr < sprv) or (mySpread > sprv and actual_spr > sprv):
            CorrectPick = 'Y'
        else:
            CorrectPick = 'N'
    else: 
        CorrectPick = 'na'
    '''
    print("Away Team: {away}, Away Average: {aavg}, Home Team: {home}, Home Avg: {havg}, Spread: {spread}, Mode: {smode}, Correct Pick: {cpick}, Sarray: {sarray}".format(away=row['v'], aavg=aavg 
                                                                                                                                , home=row['h'], havg=havg
                                                                                                                                , spread=mySpread
                                                                                                                                , smode=myModeSpread
                                                                                                                                , cpick=CorrectPick
                                                                                                                                , sarray=SpreadArray))                                                                
    '''
    
    if CorrectPick == 'Y':
        CorrectPicks += 1
    elif CorrectPick == 'N':
        IncorrectPicks += 1

print('Season: {seas}, Week: {week}, Correct Picks: {cpicks}, Incorrect Picks: {ipicks}'.format(seas=SimulateSeason, week=SimulateWeek, cpicks=CorrectPicks,ipicks=IncorrectPicks))

Season: 2018, Week: 5, Correct Picks: 8, Incorrect Picks: 6


In [16]:
### Create dataframe with min and max weeks of each season
historicalgames = pd.read_csv('data/nfl_18/SCHEDULE.csv')
testseasoncount = 5
historicalgames =  historicalgames[(historicalgames['seas'] >= 2013)]
historicalgames = historicalgames[['seas','wk']]
historicalgames['seas2'] = historicalgames['seas']
#print(historicalgames.head())

df1 = historicalgames.loc[historicalgames.groupby('seas')['wk'].idxmin()].set_index('seas')
df2 = historicalgames.loc[historicalgames.groupby('seas')['wk'].idxmax()].set_index('seas')

historicalgames = pd.concat([df1, df2], axis=1, keys=('min','max'))
historicalgames.columns = historicalgames.columns.map('_'.join)
print(historicalgames.head())


      min_wk  min_seas2  max_wk  max_seas2
seas                                      
2013       1       2013      21       2013
2014       1       2014      21       2014
2015       1       2015      21       2015
2016       1       2016      21       2016
2017       1       2017      21       2017


# Play All History

In [40]:
for index2, row2 in historicalgames.iterrows():
    for itt in range(row2['min_wk'],row2['max_wk']+1):
        
    
        global CurrentSeason, CurrentWeek
        global SimulateWeek, SimulateSeason
        global MaxWeek, MaxSeason

        CurrentWeek = 11
        CurrentSeason = 2018


        SimulateWeek = itt
        SimulateSeason = row2['min_seas2']
        print("Status: Starting Simulation of Season: {s}, Week: {w}".format(s=SimulateSeason,w=SimulateWeek))

        MaxWeek = 10
        MaxSeason = 2018

        playtypes = ['PASS','FUMBLE','INTERCPT','KOFF','PUNT','RUSH','SACK','SAFETY','TACKLE']
        predictplays = ['PASS','KOFF','PUNT','RUSH','FGXP']

        ### read ing and transform data
        playdata = readplays(SimulateSeason, SimulateWeek)
        playdata.to_csv('test.csv')
        playdata = FeatureEdit(playdata)
        #print(playdata.head(20))



        global featuresPlay, PlayModel, PlayEncoder
        global featuresPassRun, PassRunModel, PassRunEncoder
        global featuresPunt, PuntModel, PuntEncoder
        global featuresKOFF, KoffModel, KoffEncoder
        global featuresFGXP, FgxpModel, FgxpEncoder
        global featuresPlayTime, TimeModel, TimeEncoder


        
        ### drop unecessary variables
        dropvarsPlaySelect=['dseq','len','timo','timd','zone','fd','pts','tck'
         ,'sk','pen','ints','fum','saf','blk','yds_x','comp'
         ,'succ_x','spk','dfb','yds_y','succ_y','kne','kgro','knet','ktb','kr','kry'
         ,'pgro','pnet','ptb','pr','pry','pfc','pid','gid', 'wk','date','v','h'
         ,'fgxp','dist','good','playtime']

        dropvarsPassRunSuccess=['dseq','len','timo','timd','zone','fd','pts','tck'
         ,'sk','pen','ints','fum','saf','blk'
         ,'succ_x','spk','dfb','succ_y','kne','kgro','knet','ktb','kr','kry'
         ,'pgro','pnet','ptb','pr','pry','pfc','pid','gid', 'wk','date','v','h'
         ,'fgxp','dist','good','playtime']


        dropvarsPuntSuccess=['dseq','len','timo','timd','zone','fd','pts','tck'
         ,'sk','pen','ints','fum','saf','blk','yds_x','comp'
         ,'succ_x','spk','dfb','yds_y','succ_y','kne','kgro','knet','ktb','kr','kry'
         ,'pgro','ptb','pr','pry','pfc','pid','gid','wk','date','v','h'
         ,'fgxp','dist','good','playtime']

        dropvarsKoffSuccess=['dseq','len','timo','timd','zone','fd','pts','tck'
         ,'sk','pen','ints','fum','saf','blk','yds_x','comp'
         ,'succ_x','spk','dfb','yds_y','succ_y','kne','kgro','ktb','kr','kry'
         ,'pgro','pnet','ptb','pr','pry','pfc','pid','gid', 'wk','date','v','h'
         ,'fgxp','dist','good','playtime']

        dropvarsFgxpSuccess=['dseq','len','timo','timd','zone','fd','pts','tck'
         ,'sk','pen','ints','fum','saf','blk','yds_x','comp'
         ,'succ_x','spk','dfb','yds_y','succ_y','kne','kgro','knet','ktb','kr','kry'
         ,'pgro','pnet','ptb','pr','pry','pfc','pid','gid', 'wk','date','v','h'
         ,'fgxp','playtime']

        dropvarsPlayTime=['dseq','len','timo','timd','zone','fd','pts','tck'
         ,'sk','pen','ints','fum','saf','blk','yds_x','comp'
         ,'succ_x','spk','dfb','yds_y','succ_y','kne','kgro','knet','ktb','kr','kry'
         ,'pgro','pnet','ptb','pr','pry','pfc','pid','gid', 'wk','date','v','h'
         ,'fgxp']


        ### define features ###
        featuresPlay    = ['seas','off','def','qtr','sec','ptso','ptsd','dwn','ytg','yfog','day','stad','surf']
        featuresPassRun = ['seas','off','def','type','qtr','sec','ptso','ptsd','dwn','ytg','yfog','day','stad','surf']
        featuresPunt    = ['seas','off','def','qtr','sec','ptso','ptsd','dwn','ytg','yfog','day','stad','surf','player1']
        featuresKOFF    = ['seas','off','def','qtr','sec','ptso','ptsd','dwn','ytg','yfog','day','stad','surf','player1']
        featuresFGXP    = ['seas','off','def','qtr','sec','ptso','ptsd','dwn','ytg','yfog','day','stad','surf','dist','player1']
        featuresPlayTime= ['seas','off','def','type','qtr','sec','ptso','ptsd','dwn','ytg','yfog','day','stad','surf']

        print("Status: Training models")
        
        clfclassM = XGBClassifier(learning_rate =0.3,
                             n_estimators=500,
                             max_depth=3,
                             min_child_weight=1,
                             gamma=0,
                             subsample=0.8,
                             colsample_bytree=0.8,
                             objective= 'multi:softmax',
                             nthread=4,
                             scale_pos_weight=1,
                             seed=27)
        playdataPlaySelect = dropvars(playdata, dropvarsPlaySelect).copy()
        PlayModel, PlayEncoder = trainmodel(playdataPlaySelect, featuresPlay, 'type', "PlayModel", clfclassM, 'Classification')

        clfclassB = XGBClassifier(learning_rate =0.3,
                     n_estimators=500,
                     max_depth=3,
                     min_child_weight=1,
                     gamma=0,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     objective= 'binary:logistic',
                     nthread=4,
                     scale_pos_weight=1,
                     seed=27)
        playdataFGXP = dropvars(playdata, dropvarsFgxpSuccess).copy()
        playdataFGXP = playdataFGXP.loc[playdataFGXP['type'].isin(['FGXP'])]
        playdataFGXP['good'] = playdataFGXP['good'].astype(int)
        FgxpModel, FgxpEncoder = trainmodel(playdataFGXP, featuresFGXP, 'good', "FGXPModel", clfclassB, 'Classification')

        clfregress1 = XGBRegressor(max_depth = 5,
                            min_child_weight = 10,
                            subsample = 0.5,
                            colsample_bytree = 0.6,
                            objective = 'reg:linear',
                            num_estimators = 1000,
                            learning_rate = 0.3,
                            nthread=4)
        playdataPassRun = dropvars(playdata, dropvarsPassRunSuccess).copy()
        playdataPassRun = playdataPassRun.loc[playdataPassRun['type'].isin(['PASS','RUSH'])]
        playdataPassRun['YardsGain'] = playdataPassRun.apply (lambda row: yardsgain(row),axis=1)
        playdataPassRun = dropvars(playdataPassRun, ['yds_x','comp','yds_y'])
        PassRunModel, PassRunEncoder = trainmodel(playdataPassRun, featuresPassRun, 'YardsGain', "PassRunModel", clfregress1, 'Regressor')

        clfregress2 = XGBRegressor(max_depth = 5,
                        min_child_weight = 10,
                        subsample = 0.5,
                        colsample_bytree = 0.6,
                        objective = 'reg:linear',
                        num_estimators = 1000,
                        learning_rate = 0.3,
                        nthread=4)
        playdataPunt = dropvars(playdata, dropvarsPuntSuccess).copy()
        playdataPunt = playdataPunt.loc[playdataPunt['type'].isin(['PUNT'])]
        PuntModel, PuntEncoder = trainmodel(playdataPunt, featuresPunt, 'pnet', "PuntModel", clfregress2, 'Regressor')

        clfregress3 = XGBRegressor(max_depth = 5,
                        min_child_weight = 10,
                        subsample = 0.5,
                        colsample_bytree = 0.6,
                        objective = 'reg:linear',
                        num_estimators = 1000,
                        learning_rate = 0.3,
                        nthread=4)
        playdataKOFF = dropvars(playdata, dropvarsKoffSuccess).copy()
        playdataKOFF = playdataKOFF.loc[playdataKOFF['type'].isin(['KOFF'])]
        playdataKOFF = playdataKOFF[playdataKOFF['player1'].notnull()]
        KoffModel, KoffEncoder = trainmodel(playdataKOFF, featuresKOFF, 'knet', "KoffModel", clfregress3, 'Regressor')

        clfregress4 = XGBRegressor(max_depth = 5,
                        min_child_weight = 10,
                        subsample = 0.5,
                        colsample_bytree = 0.6,
                        objective = 'reg:linear',
                        num_estimators = 1000,
                        learning_rate = 0.3,
                        nthread=4)
        playdataTime = playdata[(playdata['playtime'] > 0)].copy()
        playdataTime = playdataTime.dropna(subset=['playtime'])
        playdataTime = dropvars(playdataTime, dropvarsPlayTime)
        TimeModel, TimeEncoder = trainmodel(playdataTime, featuresPlayTime, 'playtime', "TimeModel", clfregress4, 'Regressor')

        ### GET CURRENT WEEKS GAMES ###

        gamedatapredict = pd.read_csv('data/nfl_18/SCHEDULE.csv')

        gamedatapredict =  gamedatapredict[(gamedatapredict['seas']==SimulateSeason) & (gamedatapredict['wk']==SimulateWeek)]


        print("Status: Starting game simulations")
        CorrectPicks = 0
        IncorrectPicks = 0

        for index, row in gamedatapredict.iterrows():
            ### read in player data ###
            Field, Surface, HomePunter, AwayPunter, HomeQB, AwayQB, HomeKicker, AwayKicker, HomeFGKicker, AwayFGKicker = findplayers(SimulateSeason, SimulateWeek, row['h'], row['v'])

            ### define simulation stats ###
            SimulationCount = 2
            HomeWin = 0
            AwayWin = 0
            HomeScoreRun = 0
            AwayScoreRun = 0
            Tie = 0
            HomeScoreArray = []
            AwayScoreArray = []

            ### call simulator X times ###
            for itt in range(SimulationCount):
                curpct = round((itt+1)/SimulationCount*20)
                dots = '%'*curpct
                space = ' '*(20-curpct)
                print("|" + dots + space + "|  " + "Simulation: {simnum}/{simcount}".format(simnum=itt+1,simcount=SimulationCount),end="\r")
                HomeScore, AwayScore = simulate(row['h'],
                                                row['v'], 
                                                Field, 
                                                Surface, 
                                                row['day'], 
                                                HomePunter, 
                                                AwayPunter, 
                                                HomeQB, 
                                                AwayQB, 
                                                HomeKicker, 
                                                AwayKicker, 
                                                HomeFGKicker, 
                                                AwayFGKicker,
                                                itt)
                if HomeScore > AwayScore:
                    HomeWin += 1
                elif HomeScore < AwayScore:
                    AwayWin += 1
                else:
                    Tie += 1
                HomeScoreRun += HomeScore
                AwayScoreRun += AwayScore
                HomeScoreArray.append(HomeScore)
                AwayScoreArray.append(AwayScore)

            ### find spread statistics ###
            mySpread = ((AwayScoreRun-HomeScoreRun)/SimulationCount)
            SpreadArray = [AwayS - HomeS for AwayS, HomeS in zip(AwayScoreArray, HomeScoreArray)]
            myModeSpread = max(set(SpreadArray), key=SpreadArray.count)

            ### find home and away team averages ###
            havg=HomeScoreRun/SimulationCount
            aavg=AwayScoreRun/SimulationCount

            ### logic to say if spread exists ###
            if (MaxSeason > SimulateSeason) or (MaxWeek >= SimulateWeek and MaxSeason == SimulateSeason):
                SpreadData = pd.read_csv('data/nfl_18/GAME.csv')
                SpreadData2 = pd.read_csv('data/nfl_00-17/GAME.csv')
                SpreadData = pd.concat([SpreadData,SpreadData2])
                SpreadData.drop_duplicates(keep='first')
                
                SpreadData = SpreadData[(SpreadData['seas']==SimulateSeason) & (SpreadData['wk']==SimulateWeek) & (SpreadData['h']==row['h'])]

                sprv = SpreadData.iloc[0]['sprv']*-1
                actual_spr = SpreadData.iloc[0]['ptsv'] - SpreadData.iloc[0]['ptsh']
                if actual_spr == sprv:
                    CorrectPick = 'W'
                elif (mySpread <= sprv and actual_spr < sprv) or (mySpread > sprv and actual_spr > sprv):
                    CorrectPick = 'Y'
                else:
                    CorrectPick = 'N'
            else: 
                CorrectPick = 'na'
            
            print("Away Team: {away}, Away Average: {aavg}, Home Team: {home}, Home Avg: {havg}, Spread: {spread}, Mode: {smode}, Correct Pick: {cpick}, Sarray: {sarray}".format(away=row['v'], aavg=aavg 
                                                                                                                                        , home=row['h'], havg=havg
                                                                                                                                        , spread=mySpread
                                                                                                                                        , smode=myModeSpread
                                                                                                                                        , cpick=CorrectPick
                                                                                                                                        , sarray=SpreadArray)) 
            if CorrectPick == 'Y':
                CorrectPicks += 1
            elif CorrectPick == 'N':
                IncorrectPicks += 1

        print('Season: {seas}, Week: {week}, Correct Picks: {cpicks}, Incorrect Picks: {ipicks}'.format(seas=SimulateSeason, week=SimulateWeek, cpicks=CorrectPicks,ipicks=IncorrectPicks))


Status: Starting Simulation of Season: 2013, Week: 1
Status: Training models
Status: Starting game simulations
Away Team: BAL, Away Average: 33.0, Home Team: DEN, Home Avg: 11.0, Spread: 22.0, Mode: 37, Correct Pick: N, Sarray: [37, 7]
Away Team: NE, Away Average: 24.0, Home Team: BUF, Home Avg: 30.0, Spread: -6.0, Mode: -2, Correct Pick: Y, Sarray: [-2, -10]
Away Team: CIN, Away Average: 13.5, Home Team: CHI, Home Avg: 26.0, Spread: -12.5, Mode: -15, Correct Pick: W, Sarray: [-10, -15]
Away Team: MIA, Away Average: 27.5, Home Team: CLE, Home Avg: 19.0, Spread: 8.5, Mode: 11, Correct Pick: Y, Sarray: [6, 11]
Away Team: OAK, Away Average: 40.5, Home Team: IND, Home Avg: 13.0, Spread: 27.5, Mode: 10, Correct Pick: Y, Sarray: [10, 45]
Away Team: KC, Away Average: 24.0, Home Team: JAC, Home Avg: 16.0, Spread: 8.0, Mode: 3, Correct Pick: Y, Sarray: [13, 3]
Away Team: TB, Away Average: 24.0, Home Team: NYJ, Home Avg: 7.5, Spread: 16.5, Mode: 27, Correct Pick: N, Sarray: [27, 6]
Away Team: TE

Away Team: NE, Away Average: 30.5, Home Team: ATL, Home Avg: 29.0, Spread: 1.5, Mode: 0, Correct Pick: Y, Sarray: [0, 3]
Away Team: MIA, Away Average: 18.0, Home Team: NO, Home Avg: 18.5, Spread: -0.5, Mode: 0, Correct Pick: N, Sarray: [-1, 0]
Season: 2013, Week: 4, Correct Picks: 5, Incorrect Picks: 10
Status: Starting Simulation of Season: 2013, Week: 5
Status: Training models
Status: Starting game simulations
Away Team: BUF, Away Average: 26.5, Home Team: CLE, Home Avg: 21.0, Spread: 5.5, Mode: 4, Correct Pick: N, Sarray: [7, 4]
Away Team: NE, Away Average: 39.0, Home Team: CIN, Home Avg: 20.5, Spread: 18.5, Mode: 8, Correct Pick: N, Sarray: [29, 8]
Away Team: BAL, Away Average: 38.5, Home Team: MIA, Home Avg: 12.0, Spread: 26.5, Mode: 14, Correct Pick: Y, Sarray: [39, 14]
Away Team: SEA, Away Average: 30.0, Home Team: IND, Home Avg: 22.0, Spread: 8.0, Mode: 4, Correct Pick: N, Sarray: [4, 12]
Away Team: KC, Away Average: 14.5, Home Team: TEN, Home Avg: 20.5, Spread: -6.0, Mode: -8,

Status: Starting game simulations
Away Team: CIN, Away Average: 16.5, Home Team: MIA, Home Avg: 18.0, Spread: -1.5, Mode: 0, Correct Pick: Y, Sarray: [0, -3]
Away Team: KC, Away Average: 19.5, Home Team: BUF, Home Avg: 21.5, Spread: -2.0, Mode: -3, Correct Pick: N, Sarray: [-3, -1]
Away Team: ATL, Away Average: 24.0, Home Team: CAR, Home Avg: 21.0, Spread: 3.0, Mode: -15, Correct Pick: N, Sarray: [21, -15]
Away Team: SD, Away Average: 30.0, Home Team: WAS, Home Avg: 23.0, Spread: 7.0, Mode: 8, Correct Pick: N, Sarray: [8, 6]
Away Team: NO, Away Average: 19.5, Home Team: NYJ, Home Avg: 18.0, Spread: 1.5, Mode: 0, Correct Pick: Y, Sarray: [3, 0]
Away Team: TEN, Away Average: 20.0, Home Team: STL, Home Avg: 20.5, Spread: -0.5, Mode: -8, Correct Pick: N, Sarray: [7, -8]
Away Team: MIN, Away Average: 18.0, Home Team: DAL, Home Avg: 18.0, Spread: 0.0, Mode: 0, Correct Pick: Y, Sarray: [0, 0]
Away Team: TB, Away Average: 15.5, Home Team: SEA, Home Avg: 33.5, Spread: -18.0, Mode: -26, Correct 

Away Team: JAC, Away Average: 15.5, Home Team: CLE, Home Avg: 34.0, Spread: -18.5, Mode: -12, Correct Pick: N, Sarray: [-12, -25]
Away Team: TEN, Away Average: 36.5, Home Team: IND, Home Avg: 22.0, Spread: 14.5, Mode: 18, Correct Pick: N, Sarray: [11, 18]
Away Team: ARI, Away Average: 27.5, Home Team: PHI, Home Avg: 26.0, Spread: 1.5, Mode: 4, Correct Pick: W, Sarray: [4, -1]
Away Team: MIA, Away Average: 21.5, Home Team: NYJ, Home Avg: 16.5, Spread: 5.0, Mode: 3, Correct Pick: Y, Sarray: [3, 7]
Away Team: CHI, Away Average: 17.0, Home Team: MIN, Home Avg: 22.0, Spread: -5.0, Mode: -7, Correct Pick: Y, Sarray: [-7, -3]
Away Team: TB, Away Average: 22.5, Home Team: CAR, Home Avg: 24.0, Spread: -1.5, Mode: 2, Correct Pick: N, Sarray: [-5, 2]
Away Team: STL, Away Average: 12.0, Home Team: SF, Home Avg: 26.0, Spread: -14.0, Mode: -14, Correct Pick: Y, Sarray: [-14, -14]
Away Team: ATL, Away Average: 25.0, Home Team: BUF, Home Avg: 26.0, Spread: -1.0, Mode: -1, Correct Pick: Y, Sarray: [-1,

Away Team: CHI, Away Average: 16.5, Home Team: SF, Home Avg: 30.5, Spread: -14.0, Mode: -10, Correct Pick: N, Sarray: [-10, -18]
Away Team: PHI, Away Average: 36.0, Home Team: IND, Home Avg: 27.0, Spread: 9.0, Mode: 9, Correct Pick: Y, Sarray: [9, 9]
Season: 2014, Week: 2, Correct Picks: 5, Incorrect Picks: 11
Status: Starting Simulation of Season: 2014, Week: 3
Status: Training models
Status: Starting game simulations
Away Team: TB, Away Average: 12.0, Home Team: ATL, Home Avg: 31.0, Spread: -19.0, Mode: -29, Correct Pick: Y, Sarray: [-9, -29]
Away Team: SD, Away Average: 22.0, Home Team: BUF, Home Avg: 29.5, Spread: -7.5, Mode: -8, Correct Pick: N, Sarray: [-8, -7]
Away Team: TEN, Away Average: 30.0, Home Team: CIN, Home Avg: 23.0, Spread: 7.0, Mode: 11, Correct Pick: N, Sarray: [3, 11]
Away Team: BAL, Away Average: 22.5, Home Team: CLE, Home Avg: 32.0, Spread: -9.5, Mode: -7, Correct Pick: N, Sarray: [-12, -7]
Away Team: GB, Away Average: 31.0, Home Team: DET, Home Avg: 33.0, Spread

...

Away Team: GB, Away Average: 35.0, Home Team: CAR, Home Avg: 17.0, Spread: 18.0, Mode: 29, Correct Pick: N, Sarray: [29, 7]
Away Team: WAS, Away Average: 19.0, Home Team: NE, Home Avg: 34.0, Spread: -15.0, Mode: -15, Correct Pick: Y, Sarray: [-15, -15]
Away Team: TEN, Away Average: 32.0, Home Team: NO, Home Avg: 26.5, Spread: 5.5, Mode: 5, Correct Pick: Y, Sarray: [5, 6]
Away Team: MIA, Away Average: 18.5, Home Team: BUF, Home Avg: 19.0, Spread: -0.5, Mode: 0, Correct Pick: N, Sarray: [0, -1]
Away Team: STL, Away Average: 6.0, Home Team: MIN, Home Avg: 26.5, Spread: -20.5, Mode: -21, Correct Pick: Y, Sarray: [-21, -20]
Away Team: JAC, Away Average: 14.0, Home Team: NYJ, Home Avg: 21.0, Spread: -7.0, Mode: -20, Correct Pick: Y, Sarray: [6, -20]
Away Team: OAK, Away Average: 16.5, Home Team: PIT, Home Avg: 24.5, Spread: -8.0, Mode: -6, Correct Pick: N, Sarray: [-6, -10]
Away Team: NYG, Away Average: 37.0, Home Team: TB, Home Avg: 22.5, Spread: 14.5, Mode: 10, Correct Pick: Y, Sarray: [10

Away Team: NYJ, Away Average: 16.5, Home Team: BUF, Home Avg: 26.0, Spread: -9.5, Mode: -14, Correct Pick: Y, Sarray: [-5, -14]
Away Team: NE, Away Average: 47.0, Home Team: MIA, Home Avg: 16.5, Spread: 30.5, Mode: 22, Correct Pick: N, Sarray: [22, 39]
Away Team: NO, Away Average: 38.0, Home Team: ATL, Home Avg: 34.0, Spread: 4.0, Mode: 8, Correct Pick: Y, Sarray: [8, 0]
Away Team: BAL, Away Average: 25.5, Home Team: CIN, Home Avg: 22.0, Spread: 3.5, Mode: 10, Correct Pick: Y, Sarray: [10, -3]
Away Team: PIT, Away Average: 32.5, Home Team: CLE, Home Avg: 32.0, Spread: 0.5, Mode: 4, Correct Pick: N, Sarray: [-3, 4]
Away Team: JAC, Away Average: 11.0, Home Team: HOU, Home Avg: 27.0, Spread: -16.0, Mode: -8, Correct Pick: Y, Sarray: [-8, -24]
Away Team: TEN, Away Average: 13.5, Home Team: IND, Home Avg: 30.5, Spread: -17.0, Mode: -15, Correct Pick: Y, Sarray: [-19, -15]
Away Team: WAS, Away Average: 23.5, Home Team: DAL, Home Avg: 29.5, Spread: -6.0, Mode: -8, Correct Pick: N, Sarray: [-8

Away Team: PHI, Away Average: 24.0, Home Team: CHI, Home Avg: 26.5, Spread: -2.5, Mode: -4, Correct Pick: Y, Sarray: [-1, -4]
Season: 2016, Week: 2, Correct Picks: 7, Incorrect Picks: 9
Status: Starting Simulation of Season: 2016, Week: 3
Status: Training models
Status: Starting game simulations
Away Team: HOU, Away Average: 16.5, Home Team: NE, Home Avg: 21.0, Spread: -4.5, Mode: -7, Correct Pick: Y, Sarray: [-7, -2]
Away Team: ARI, Away Average: 12.0, Home Team: BUF, Home Avg: 19.5, Spread: -7.5, Mode: 3, Correct Pick: Y, Sarray: [3, -18]
Away Team: BAL, Away Average: 6.0, Home Team: JAC, Home Avg: 3.0, Spread: 3.0, Mode: 0, Correct Pick: Y, Sarray: [0, 6]
Away Team: CLE, Away Average: 18.5, Home Team: MIA, Home Avg: 24.0, Spread: -5.5, Mode: -7, Correct Pick: Y, Sarray: [-4, -7]
Away Team: DEN, Away Average: 6.0, Home Team: CIN, Home Avg: 7.5, Spread: -1.5, Mode: 0, Correct Pick: Y, Sarray: [0, -3]
Away Team: DET, Away Average: 36.0, Home Team: GB, Home Avg: 26.0, Spread: 10.0, Mode

Away Team: BUF, Away Average: 9.5, Home Team: CIN, Home Avg: 8.0, Spread: 1.5, Mode: 0, Correct Pick: Y, Sarray: [0, 3]
Away Team: CHI, Away Average: 19.0, Home Team: NYG, Home Avg: 31.5, Spread: -12.5, Mode: -21, Correct Pick: N, Sarray: [-4, -21]
Away Team: JAC, Away Average: 11.5, Home Team: DET, Home Avg: 26.0, Spread: -14.5, Mode: -4, Correct Pick: Y, Sarray: [-4, -25]
Away Team: PIT, Away Average: 8.5, Home Team: CLE, Home Avg: 21.0, Spread: -12.5, Mode: -14, Correct Pick: N, Sarray: [-14, -11]
Away Team: TB, Away Average: 0.0, Home Team: KC, Home Avg: 20.5, Spread: -20.5, Mode: -21, Correct Pick: N, Sarray: [-21, -20]
Away Team: TEN, Away Average: 13.0, Home Team: IND, Home Avg: 27.5, Spread: -14.5, Mode: -8, Correct Pick: Y, Sarray: [-21, -8]
Away Team: MIA, Away Average: 17.5, Home Team: LA, Home Avg: 6.5, Spread: 11.0, Mode: 10, Correct Pick: Y, Sarray: [12, 10]
Away Team: NE, Away Average: 32.5, Home Team: SF, Home Avg: 8.0, Spread: 24.5, Mode: 27, Correct Pick: Y, Sarray: [

KeyboardInterrupt: 